### Use case notebook: schedule CCB team meetings ###
Date: 05/28/2025

In [1]:
import os
import glob
import numpy as np
from collections import deque

# There is a warning in the timeboard library that we want to suppress here
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
from pathlib import Path
import time
import timeboard as tb
import timeboard.calendars.US as US
import datetime
import holidays

# Appearance of the Notebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Import the package
%load_ext autoreload
%autoreload 2
import cadence
from cadence.utils import FileOP
from cadence.mscheduler import Meetings
from cadence.mscheduler import cyclic_permutate
# print(f'Package version: {cadence.__version__}')

### Meeting participants and settings ###

In [2]:
# Docker drive
data_dir = os.environ.get('DATA')

# Output data to the root of the repo
output_dir = os.environ.get('HOME')

group_members_file_name = 'ccb_members_2025.xlsx'
url_base = 'https://dsets.s3.us-east-1.amazonaws.com'
url = os.path.join(url_base, group_members_file_name)

# Download file
member_file = FileOP().download_from_url(url=url, download_dir=data_dir)
tm_raw = pd.read_excel(member_file)
display(tm_raw)

ccb_members_2025.xlsx: 16.4kB [00:00, 128kB/s]                             

Created .xlsx file.


,name,email,group
0,Andreas,andreas_werdich@hms.harvard.edu,ai
1,Gerald,gerald_kiwanuka@hms.harvard.edu,ai
2,Nathan,nathan_palmer@hms.harvard.edu,ai
3,Nidia,nidia_bucarellisanchez@hms.harvard.edu,ai
4,Alex,alex_pickering@hms.harvard.edu,cbio
5,Andrew,andrew_ghazi@hms.harvard.edu,cbio
6,Anthony,anthony-alexander_christidis@hms.harvard.edu,cbio
7,Grey,grey_kuling@hms.harvard.edu,cbio
8,Ludwig,ludwig_geistlinger@hms.harvard.edu,cbio
9,Tram,tram_nguyen@hms.harvard.edu,cbio


### Cleanup of the spreadsheet ###
Just in case that was not done. To provide a consistent look.

In [3]:
email_col = 'email'
name_col = 'name'
group_col = 'group'

# Lets do some cleanup of the spreadsheet
# small letters for the column names
tm = tm_raw.copy()
tm.columns = [col.lower() for col in tm.columns]

# small letters for emails and group, first letter capitalize for names
tm[email_col] = tm[email_col].str.lower()
tm[group_col] = tm[group_col].str.lower()
tm[name_col] = tm[name_col].str.title()

# Sort by group and then name
tm = tm.sort_values(by=['group', 'name'], ascending=True).reset_index(drop=True)

# Mark names that are not presenting
rm_name = ['nathan', 'ludwig']
rm_name = [nm.title() for nm in rm_name]
tm = tm.assign(presenting=True)
tm.loc[tm['name'].isin(rm_name), 'presenting'] = False

# save the name ist as a csv_file
presenter_list_file_name = f'{os.path.splitext(group_members_file_name)[0]}_list.csv'
presenter_list_file = os.path.join(data_dir, presenter_list_file_name)
tm.to_csv(presenter_list_file, index=False)
display(tm)

,name,email,group,presenting
0,Andreas,andreas_werdich@hms.harvard.edu,ai,True
1,Gerald,gerald_kiwanuka@hms.harvard.edu,ai,True
2,Nathan,nathan_palmer@hms.harvard.edu,ai,False
3,Nidia,nidia_bucarellisanchez@hms.harvard.edu,ai,True
4,Alex,alex_pickering@hms.harvard.edu,cbio,True
5,Andrew,andrew_ghazi@hms.harvard.edu,cbio,True
6,Anthony,anthony-alexander_christidis@hms.harvard.edu,cbio,True
7,Grey,grey_kuling@hms.harvard.edu,cbio,True
8,Ludwig,ludwig_geistlinger@hms.harvard.edu,cbio,False
9,Tram,tram_nguyen@hms.harvard.edu,cbio,True


In [4]:
# Remove the names that we do not want in the schedule
presenters_df = tm.loc[tm['presenting']==True]
display(presenters_df)

,name,email,group,presenting
0,Andreas,andreas_werdich@hms.harvard.edu,ai,True
1,Gerald,gerald_kiwanuka@hms.harvard.edu,ai,True
3,Nidia,nidia_bucarellisanchez@hms.harvard.edu,ai,True
4,Alex,alex_pickering@hms.harvard.edu,cbio,True
5,Andrew,andrew_ghazi@hms.harvard.edu,cbio,True
6,Anthony,anthony-alexander_christidis@hms.harvard.edu,cbio,True
7,Grey,grey_kuling@hms.harvard.edu,cbio,True
9,Tram,tram_nguyen@hms.harvard.edu,cbio,True
10,Tyrone,tyrone_lee@hms.harvard.edu,cbio,True


### Shuffle the data and create a presenter list ###

In [5]:
# Let's do a random shuffling of the data frame
random_state = 234
presenters_df_shuffled = presenters_df.sample(frac=1, random_state=random_state)

# Get the names and the groups from the new data frame
name_list = list(presenters_df_shuffled['name'].values)
group_list = list(presenters_df_shuffled['group'].values)

# Instantiate the Meetings class with the list of names and groups from the data frame
meet = Meetings(name_list=name_list, group_list=group_list)

# We create a presenter list by merging the groups so that we have a member 
# from a different group presenting each time
name_sequence = meet.create_name_sequence()
# Rotate the sequence (start with a specific name)
rotated_sequence = cyclic_permutate(name_sequence, name='Grey')
# updated_sequence = ['Grey', 'Andrew','Alex', 'Gerald', 'Tram', 'Andreas', 'Anthony', 'Nidia', 'Tyrone']
updated_sequence = ['Grey', 'Andrew','Alex', 'Nidia', 'Gerald', 'Tram', 'Andreas', 'Anthony', 'Tyrone']
new_presenter_list = meet.create_name_sequence(name_sequence=updated_sequence, merge_groups=False)
display(meet.name_df)

,name,group
0,Grey,cbio
1,Andrew,cbio
2,Alex,cbio
3,Nidia,ai
4,Gerald,ai
5,Tram,cbio
6,Andreas,ai
7,Anthony,cbio
8,Tyrone,cbio


### Create the meeting schedule ###

In [10]:
start_date = '2025-05-28'
end_date = '2025-08-27'

cal = meet.create_timeboard(start_date=start_date, 
                            end_date=end_date,
                            start_name='Nidia')
skip_date = '2025-05-28'
skip_name = 'No Meeting'
skip_comment = 'No Meeting'

cal = meet.skip_date(cal_df=cal, 
                     date=skip_date, 
                     comment=skip_comment,
                     name=skip_name)

skip_date = '2025-06-04'
skip_name = 'Andreas, Nathan'
skip_comment = 'AI resources presentation'

cal = meet.skip_date(cal_df=cal, 
                     date=skip_date, 
                     comment=skip_comment,
                     name=skip_name)

skip_date = '2025-06-18'
skip_name = 'Nidia'
skip_comment = 'ARI transcription project'

cal = meet.skip_date(cal_df=cal, 
                     date=skip_date, 
                     comment=skip_comment,
                     name=skip_name)

skip_date = '2025-06-25'
skip_name = 'No Meeting'
skip_comment = 'No Meeting'

cal = meet.skip_date(cal_df=cal, 
                     date=skip_date, 
                     comment=skip_comment,
                     name=skip_name)

skip_date = '2025-07-02'
skip_name = "Cookout at Nathan's house"
skip_comment = "Cookout at Nathan's house"

cal = meet.skip_date(cal_df=cal, 
                     date=skip_date, 
                     comment=skip_comment,
                     name=skip_name)

display(cal)

,date,name,group,holiday,comment
0,2025-05-28,No Meeting,ai,False,No Meeting
1,2025-06-04,"Andreas, Nathan",ai,False,AI resources presentation
2,2025-06-11,Nidia,ai,False,None
3,2025-06-18,Nidia,ai,False,ARI transcription project
4,2025-06-25,No Meeting,ai,False,No Meeting
5,2025-07-02,Cookout at Nathan's house,ai,False,Cookout at Nathan's house
6,2025-07-09,Gerald,ai,False,None
7,2025-07-16,Tram,cbio,False,None
8,2025-07-23,Andreas,ai,False,None
9,2025-07-30,Anthony,cbio,False,None


In [11]:
# Save the presentation schedule
save_cols = ['date', 'name', 'comment']
schedule_name = f'ccb_presentations_2025_06.csv'
schedule_file = os.path.join(output_dir, schedule_name)
cal[save_cols].to_csv(schedule_file, index=False)
display(cal[save_cols])
print(schedule_file)

,date,name,comment
0,2025-05-28,No Meeting,No Meeting
1,2025-06-04,"Andreas, Nathan",AI resources presentation
2,2025-06-11,Nidia,None
3,2025-06-18,Nidia,ARI transcription project
4,2025-06-25,No Meeting,No Meeting
5,2025-07-02,Cookout at Nathan's house,Cookout at Nathan's house
6,2025-07-09,Gerald,None
7,2025-07-16,Tram,None
8,2025-07-23,Andreas,None
9,2025-07-30,Anthony,None


/app/ccb_presentations_2025_06.csv
